In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import numpy as np
import datetime
from bs4 import BeautifulSoup
import requests
import psycopg2
from datetime import datetime




In [2]:
datetime.now().strftime('%m/%d/%Y')
today = datetime.now().strftime('%m/%d/%Y')


In [3]:
url = "https://finance.yahoo.com/most-active"
url2 = "https://finance.yahoo.com/gainers"
url3 = "https://finance.yahoo.com/losers"

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.content, 'lxml')

In [4]:
active_stocks = pd.read_html(url)
active_stocks = active_stocks[0]
#active_table.head()
active_stocks.rename(columns={'Volume':'Volume_Millions'}, inplace=True)
active_stocks['Volume_Millions'] = active_stocks['Volume_Millions'].str.replace(r'M', '').astype(float)
active_stocks.rename(columns={'Avg Vol (3 month)':'Avg_3_Month_Vol_Millions'}, inplace=True)
active_stocks['Avg_3_Month_Vol_Millions'] = active_stocks['Avg_3_Month_Vol_Millions'].str.replace(r'M', '').astype(float)
active_stocks['Date'] = pd.to_datetime('today').strftime("%m/%d/%Y")
del active_stocks['PE Ratio (TTM)'] 
del active_stocks['52 Week Range']
active_stocks.head()

,Symbol,Name,Price (Intraday),Change,% Change,Volume_Millions,Avg_3_Month_Vol_Millions,Market Cap,Date
0,NIO,NIO Limited,48.95,2.23,+4.77%,131.225,169.897,76.309B,12/21/2020
1,AAPL,Apple Inc.,128.23,1.57,+1.24%,121.252,120.209,2.18T,12/21/2020
2,BAC,Bank of America Corporation,29.74,1.07,+3.73%,87.342,58.298,257.274B,12/21/2020
3,AAL,American Airlines Group Inc.,16.10,-0.41,-2.48%,87.244,75.650,9.833B,12/21/2020
4,GE,General Electric Company,10.76,-0.05,-0.46%,85.819,107.213,94.256B,12/21/2020


In [5]:
gain_table = pd.read_html(url2)
gain_table = gain_table[0]
gain_table.rename(columns={'Volume':'Volume_Millions'}, inplace=True)
gain_table['Volume_Millions'] = gain_table['Volume_Millions'].str.replace(r'M', '').astype(float)
gain_table.rename(columns={'Avg Vol (3 month)':'Avg_3_Month_Vol_Millions'}, inplace=True)
gain_table['Avg_3_Month_Vol_Millions'] = gain_table['Avg_3_Month_Vol_Millions'].str.replace(r'M', '').astype(float)
gain_table['Date'] = pd.to_datetime('today').strftime("%m/%d/%Y")
del gain_table['PE Ratio (TTM)'] 
del gain_table['52 Week Range']
gain_table.head()

,Symbol,Name,Price (Intraday),Change,% Change,Volume_Millions,Avg_3_Month_Vol_Millions,Market Cap,Date
0,EDIT,"Editas Medicine, Inc.",83.66,19.25,+29.89%,10.629,1.427,5.215B,12/21/2020
1,RP,"RealPage, Inc.",87.33,19.50,+28.75%,16.432,734564.000,8.895B,12/21/2020
2,QS,QuantumScape Corporation,94.90,21.17,+28.71%,26.559,5.771,34.445B,12/21/2020
3,AGIO,"Agios Pharmaceuticals, Inc.",42.62,9.41,+28.33%,5.740,765778.000,2.952B,12/21/2020
4,LAZR,Luminar Technologies Inc,35.15,7.53,+27.26%,36.964,7.903,11.386B,12/21/2020


In [6]:
loser_table = pd.read_html(url3)
loser_table = loser_table[0]
loser_table.rename(columns={'Volume':'Volume_Millions'}, inplace=True)
loser_table['Volume_Millions'] = loser_table['Volume_Millions'].str.replace(r'M', '').astype(float)
loser_table.rename(columns={'Avg Vol (3 month)':'Avg_3_Month_Vol_Millions'}, inplace=True)
loser_table['Avg_3_Month_Vol_Millions'] = loser_table['Avg_3_Month_Vol_Millions'].str.replace(r'M', '').astype(float)
loser_table['Date'] = pd.to_datetime('today').strftime("%m/%d/%Y")
del loser_table['PE Ratio (TTM)'] 
del loser_table['52 Week Range']
loser_table.head()

,Symbol,Name,Price (Intraday),Change,% Change,Volume_Millions,Avg_3_Month_Vol_Millions,Market Cap,Date
0,LKNCY,Luckin Coffee Inc.,8.20,-1.22,-12.95%,16.528,4.659,2.076B,12/21/2020
1,BABWF,"International Consolidated Airlines Group, S.A.",1.97,-0.22,-10.05%,118202.000,75715.000,9.477B,12/21/2020
2,FGEN,"FibroGen, Inc.",40.01,-3.96,-9.01%,3.410,678618.000,3.641B,12/21/2020
3,KPCPY,Kasikornbank Public Company Limited,14.66,-1.38,-8.58%,20450.000,62845.000,8.497B,12/21/2020
4,TTM,Tata Motors Limited,11.19,-0.99,-8.13%,2.615,1.609,7.449B,12/21/2020


In [7]:
SQL_USERNAME = "postgres"
SQL_PASSWORD = "password"
SQL_IP = "35.238.220.1"
PORT = 5432
DATABASE = "postgres"

In [8]:
connection_string = f"postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

In [9]:
inspector = inspect(engine)
inspector.get_table_names()

['gain_table', 'loser_table', 'ticker_data', 'active_stocks']

In [10]:
active_stocks.columns

Index(['Symbol', 'Name', 'Price (Intraday)', 'Change', '% Change',
       'Volume_Millions', 'Avg_3_Month_Vol_Millions', 'Market Cap', 'Date'],
      dtype='object')

In [13]:
conn = engine.connect()
conn.execute(f"delete from active_stocks where date='{today}'")
active_tbl_pg = active_stocks.copy()
active_tbl_pg.columns = ['Symbol', 'Name', 'Price_Intraday', 'Change', 'PCT_Change', 'Volume_Millions', 
                         'Avg_3_Month_Vol_Millions', 'Market_Cap', 'Date']
active_tbl_pg.columns = [x.lower() for x in active_tbl_pg.columns]

active_tbl_pg.to_sql("active_stocks", engine, if_exists='append', index=False, method='multi')

In [14]:
gain_table.columns

Index(['Symbol', 'Name', 'Price (Intraday)', 'Change', '% Change',
       'Volume_Millions', 'Avg_3_Month_Vol_Millions', 'Market Cap', 'Date'],
      dtype='object')

In [15]:
conn = engine.connect()
conn.execute(f"delete from gain_table where date='{today}'")


gain_tbl_pg = gain_table.copy()
gain_tbl_pg.columns = ['Symbol', 'Name', 'Price_Intraday', 'Change', 'PCT_Change', 'Volume_Millions',
       'Avg_3_Month_Vol_Millions', 'Market_Cap', 'Date']
gain_tbl_pg.columns = [x.lower() for x in gain_tbl_pg.columns]

gain_tbl_pg.to_sql("gain_table", engine, if_exists='append', index=False, method='multi')

In [16]:
loser_table.columns

Index(['Symbol', 'Name', 'Price (Intraday)', 'Change', '% Change',
       'Volume_Millions', 'Avg_3_Month_Vol_Millions', 'Market Cap', 'Date'],
      dtype='object')

In [17]:

conn = engine.connect()
conn.execute(f"delete from loser_table where date='{today}'")
loser_tbl_pg = loser_table.copy()
loser_tbl_pg.columns = ['Symbol', 'Name', 'Price_Intraday', 'Change', 'PCT_Change', 'Volume_Millions',
       'Avg_3_Month_Vol_Millions', 'Market_Cap', 'Date']
loser_tbl_pg.columns = [x.lower() for x in loser_tbl_pg.columns]

loser_tbl_pg.to_sql("loser_table", engine, if_exists='append',  index=False, method='multi')

In [18]:
# conn = psycopg2.connect("dbname=postgres user=postgres password=password")
# cur.execute("SELECT * FROM users;")
# users = cur.fetchall()
# conn.close()

In [19]:
filepath = "nasdaq_screener.csv"
ticker_data = pd.read_csv(filepath)
ticker_data['% Change'] = ticker_data['% Change'].str.replace(r'%', '').astype(float)
ticker_data.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$117.78,-1.5200,-1.27,3.614074e+10,United States,1999.0,2081968,Capital Goods,Biotechnology: Laboratory Analytical Instruments
1,AA,Alcoa Corporation Common Stock,$22.11,0.1000,0.45,4.110903e+09,NaN,2016.0,3868805,Basic Industries,Aluminum
2,AACG,ATA Creativity Global American Depositary Shares,$1.20,-0.0002,-0.02,3.762060e+07,China,NaN,12050,Consumer Services,Other Consumer Services
3,AACQ,Artius Acquisition Inc. Class A Common Stock,$10.40,-0.0300,-0.29,9.418500e+08,United States,2020.0,944842,Finance,Business Services
4,AACQU,Artius Acquisition Inc. Unit,$11.00,0.0200,0.18,0.000000e+00,United States,2020.0,181523,Finance,Business Services


In [20]:
ticker_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7245 entries, 0 to 7244
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      7245 non-null   object 
 1   Name        7245 non-null   object 
 2   Last Sale   6411 non-null   object 
 3   Net Change  6411 non-null   float64
 4   % Change    6401 non-null   float64
 5   Market Cap  6309 non-null   float64
 6   Country     6155 non-null   object 
 7   IPO Year    3757 non-null   float64
 8   Volume      7245 non-null   int64  
 9   Sector      5670 non-null   object 
 10  Industry    5670 non-null   object 
dtypes: float64(4), int64(1), object(6)
memory usage: 622.7+ KB


In [22]:
conn = engine.connect()
conn.execute(f"truncate table ticker_data")
ticker_data_tbl_pg = ticker_data.copy()
ticker_data_tbl_pg.columns = ['Symbol', 'Name', 'Last_Sale', 'Net_Change', 'PCT_Change', 'Market_Cap',
     'Country', 'IPO_Year', 'Volume', 'Sector', 'Industry']

ticker_data_tbl_pg.columns = [x.lower() for x in ticker_data_tbl_pg.columns]
ticker_data_tbl_pg.to_sql('ticker_data', engine, if_exists='append', index=False, method='multi')